In [1]:
# if you run into an error you may need to run this twice
!pip install boto3
!pip install aws-kinesis-agg
!pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 30.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 1.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 7.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botoc

**Make sure to create an `aws.cfg` file and upload it here**

aws.cfg example:

```
[AWS]
aws_access_key_id = youraccesskey
aws_secret_access_key = yoursecretkey
region_name=us-east-1
```
Make sure to change the values to match yours.

In [2]:
import boto3
import json
import time
import pandas as pd
import configparser

# Read AWS credentials from config file
config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key_id = config['AWS']['aws_access_key_id']
aws_secret_access_key = config['AWS']['aws_secret_access_key']
region_name = config['AWS']['region_name']




In [3]:
# Initialize the boto3 client with credentials from config file
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)


In [4]:
def produce(stream_name, data, partition_key):
    try:
        # Convert timestamps to strings
        for key in data:
            if isinstance(data[key], pd.Timestamp):
                data[key] = data[key].isoformat()
                # using the put_record method to push the stream
        response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey=partition_key
        )
        return response
    except Exception as e:
        print(f"Error producing record: {e}")

# this function takes 5 records at a time and streams every 2-seconds
def stream_data(df, stream_name):
    for i in range(0, len(df), 5):
        records = df.iloc[i:i+5].to_dict(orient='records')
        for record in records:
            partition_key = str(record['tpep_pickup_datetime'])  # Use a valid column as the partition key

            # call the produce function
            produce(stream_name, record, partition_key)
        print(f"Sent {len(records)} records to Kinesis")
        time.sleep(2)


In [6]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.9 MB/s eta 0:00:00:00:0100:01


In [7]:
if __name__ == "__main__":
    stream_name = 'input-stream'
    # Reading the yellow_taxis parquet file and stream it
    df = pd.read_parquet('s3://techcatalyst-public/yellow_tripdata_2024-01.parquet',
                     storage_options={
                      'key': aws_access_key_id,
                     'secret': aws_secret_access_key,
                     })
    stream_data(df, stream_name)

Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis
Sent 5 records to Kinesis


KeyboardInterrupt: 